<a href="https://colab.research.google.com/github/timothyafolami/project-1/blob/main/NIRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import normalize, MinMaxScaler
import scipy
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.losses import SparseCategoricalCrossentropy
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
# from scipy import scipy.signal.savgol_filter as sg

In [4]:
from scipy import signal

In [5]:
!pip install pyspectra

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for spc-spectra: filename=spc_spectra-0.4.0-py3-none-any.whl size=10363 sha256=2a86942626f334e59ca72aff786d1469495e84f97f1a1970db5ee5c9950df38a
  Stored in directory: /root/.cache/pip/wheels/40/be/7b/f77433f120fa2d527a0d3a16ec79658f263236b139ac151177
Successfully built spc-spectra


### Loading and reading the data.

In [6]:
df = pd.read_excel('NewData-V2.xlsx')
df.head()

,Fruit Type,Fruit,Daypoint,Side,650,651,652,653,654,655,...,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050
0,Apple1,Apple,Day0,Right,0.643975,0.636900,0.612648,0.596753,0.582198,0.562480,...,0.024283,0.009159,0.012014,0.016988,0.028033,0.028408,0.025564,0.022167,0.026624,0.031955
1,Apple1,Apple,Day0,Right,0.995426,0.949976,0.928001,0.871681,0.814014,0.776765,...,0.108691,0.104369,0.102381,0.102553,0.113067,0.131942,0.132410,0.125606,0.126971,0.146244
2,Apple1,Apple,Day0,Right,0.748004,0.773538,0.777398,0.757476,0.734280,0.721468,...,0.083332,0.078086,0.072635,0.077132,0.087525,0.096934,0.086114,0.074600,0.074297,0.088752
3,Apple1,Apple,Day0,Right,0.816814,0.822823,0.797048,0.772206,0.729269,0.695100,...,0.065693,0.064226,0.060804,0.063224,0.075397,0.086232,0.083173,0.072525,0.080899,0.102872
4,Apple1,Apple,Day0,Right,0.836698,0.805913,0.776527,0.746743,0.729695,0.710251,...,0.064154,0.072661,0.071913,0.068891,0.083740,0.095423,0.091856,0.075014,0.072472,0.073064


In [7]:
# SVN function



def snv(input_data):
  
    # Define a new array and populate it with the corrected data  
    output_data = np.zeros_like(input_data)
    for i in range(input_data.shape[0]):
 
        # Apply correction
        output_data[i,:] = (input_data[i,:] - np.mean(input_data[i,:])) / np.std(input_data[i,:])
 
    return output_data

In [8]:
check = ['Fruit Type', 'Fruit', 'Daypoint']
for i in check:
    print(df[i].value_counts())

Apple1     100
Orange1    100
Mango1     100
Mango2     100
Orange2    100
Mango3      99
Orange3     60
Mango4      60
Orange4     40
Mango5      20
Apple2      20
Apple3      20
Mango6      20
Apple4      20
Orange5     20
Name: Fruit Type, dtype: int64
Mangoe    399
Orange    320
Apple     160
Name: Fruit, dtype: int64
Day0     199
Day5     180
Day10    160
Day15    120
Day20    120
Day25    100
Name: Daypoint, dtype: int64


In [9]:
check = ['Fruit Type', 'Fruit', 'Daypoint', 'Side']
for i in check:
    print(df[i].unique())

['Apple1' 'Orange1' 'Mango1' 'Mango2' 'Orange2' 'Mango5' 'Apple2' 'Apple3'
 'Mango6' 'Orange3' 'Orange4' 'Mango3' 'Mango4' 'Apple4' 'Orange5']
['Apple' 'Orange' 'Mangoe']
['Day0' 'Day5' 'Day10' 'Day15' 'Day20' 'Day25']
['Right' 'Left']


In [10]:
# Splitting the data to train and test set.

from sklearn.model_selection import train_test_split

train_set, val_set = train_test_split(df, test_size=0.2 ,random_state=42)
train_set.shape, val_set.shape

((703, 405), (176, 405))

In [11]:
train_wave = train_set.drop(check, axis=1)
val_wave = val_set.drop(check, axis=1)
train_wave.head()

,650,651,652,653,654,655,656,657,658,659,...,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050
290,0.717442,0.720352,0.651176,0.623778,0.594663,0.609433,0.590414,0.594175,0.593588,0.602699,...,0.088614,0.095338,0.088431,0.079580,0.075111,0.081566,0.071201,0.064746,0.072303,0.078041
685,0.704957,0.703461,0.687085,0.647752,0.629148,0.627250,0.631064,0.628375,0.607656,0.599794,...,0.101371,0.101252,0.102329,0.102916,0.101635,0.104001,0.101840,0.104203,0.111256,0.114307
547,1.041835,1.152231,1.181060,1.103420,0.967680,0.923223,0.943483,1.011206,1.022966,1.046402,...,0.230261,0.236457,0.242061,0.243443,0.243951,0.239019,0.238239,0.234902,0.238358,0.245942
834,0.481216,0.503996,0.516887,0.495927,0.506037,0.509924,0.518859,0.512957,0.518841,0.536486,...,0.139091,0.144048,0.147357,0.147208,0.148288,0.152292,0.154349,0.150241,0.153048,0.142493
798,0.818876,0.803494,0.759452,0.719022,0.707352,0.691306,0.683509,0.713203,0.734064,0.741874,...,0.299068,0.303179,0.306244,0.316072,0.323958,0.329405,0.336348,0.338345,0.332627,0.315982


### Normalizing the Data.

In [12]:
# Normalizing with normalizer and minmax separately.



train_norm = normalize(train_wave)
val_norm = normalize(val_wave)

minmax = MinMaxScaler()
train_minmax = minmax.fit_transform(train_wave)
val_minmax = minmax.transform(val_wave)
val_minmax

array([[0.68383488, 0.54579087, 0.56343748, ..., 0.26414429, 0.24383132,
        0.25188498],
       [0.2025773 , 0.1411086 , 0.16828459, ..., 0.57366625, 0.57066508,
        0.58382064],
       [0.48244706, 0.3305853 , 0.36355621, ..., 0.77583817, 0.81124604,
        0.82444891],
       ...,
       [0.56365117, 0.40344972, 0.45871031, ..., 0.68531392, 0.70648489,
        0.70595118],
       [0.2252281 , 0.15209709, 0.17267489, ..., 0.56273677, 0.5060533 ,
        0.48958689],
       [0.10444076, 0.074012  , 0.09166486, ..., 0.29556445, 0.2795801 ,
        0.26982489]])

In [13]:
# SNV preprocessing
train_snv = snv(train_wave.to_numpy())
val_snv = snv(val_wave.to_numpy())
val_snv

array([[ 2.03034523,  2.31647062,  2.00811282, ..., -0.87456814,
        -0.89181902, -0.88358997],
       [ 1.32148839,  1.32853456,  1.33602014, ..., -0.98993104,
        -0.99783323, -1.01061539],
       [ 2.76726517,  2.66213446,  2.41039912, ..., -0.88387082,
        -0.77254613, -0.82049942],
       ...,
       [ 2.62437997,  2.66826452,  2.55846295, ..., -0.57291442,
        -0.52548643, -0.57981315],
       [ 1.63462032,  1.57843327,  1.49692693, ..., -0.9498237 ,
        -1.16444495, -1.27642854],
       [ 1.8080119 ,  1.8501307 ,  1.87976073, ..., -0.78992513,
        -0.82250049, -0.87254637]])

In [14]:
# Savitzky–Golay

train_savgol = signal.savgol_filter(train_wave, window_length=175, polyorder=10)
val_savgol = scipy.signal.savgol_filter(val_wave, window_length=175, polyorder=10)
val_savgol

array([[1.15622976, 1.1337334 , 1.11193107, ..., 0.01844328, 0.01664885,
        0.0146678 ],
       [0.49354723, 0.51528477, 0.53143898, ..., 0.18632545, 0.18223161,
        0.1776794 ],
       [0.8331238 , 0.81827733, 0.80448254, ..., 0.30047286, 0.30314963,
        0.3062569 ],
       ...,
       [0.96996267, 0.94472433, 0.92319719, ..., 0.24708966, 0.25051553,
        0.25387248],
       [0.51794852, 0.52527897, 0.52992191, ..., 0.16077452, 0.15546137,
        0.14981471],
       [0.38115001, 0.39232185, 0.39956114, ..., 0.02316936, 0.02276797,
        0.02237584]])

In [15]:
# MODELS

from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

In [ ]:
from sklearn.preprocessing import LabelEncoder

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(train_set['Fruit Type'])
encoded_Y = encoder.transform(train_set['Fruit Type'])
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

val_encoder = encoder.transform(val_set['Fruit Type'])
val_encoder_y = np_utils.to_categorical(val_encoder)
val_encoder_y

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

### Random Forest CLassifier

In [ ]:
rf_clf = RandomForestClassifier(random_state=42)

# Normalizer
rf_clf.fit(train_norm, train_set['Fruit Type'])
norm_pred = rf_clf.predict(val_norm)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.9318181818181818
              precision    recall  f1-score   support

      Apple1       0.83      1.00      0.90        19
      Apple2       1.00      1.00      1.00         3
      Apple3       1.00      1.00      1.00         2
      Apple4       1.00      1.00      1.00         4
      Mango1       0.88      0.74      0.80        19
      Mango2       0.73      0.94      0.82        17
      Mango3       1.00      1.00      1.00        18
      Mango4       1.00      0.83      0.91        12
      Mango5       1.00      1.00      1.00         2
      Mango6       1.00      1.00      1.00         5
     Orange1       1.00      0.96      0.98        27
     Orange2       1.00      1.00      1.00        20
     Orange3       1.00      0.80      0.89        15
     Orange4       1.00      1.00      1.00         8
     Orange5       1.00      1.00      1.00         5

    accuracy                           0.93       176
   macro avg       0.96      0.95      0.95       176
weighte

In [ ]:
# Minmax Normalizer

rf_clf.fit(train_minmax, train_set['Fruit Type'])
norm_pred = rf_clf.predict(val_minmax)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.9147727272727273
              precision    recall  f1-score   support

      Apple1       0.95      0.95      0.95        19
      Apple2       1.00      1.00      1.00         3
      Apple3       1.00      1.00      1.00         2
      Apple4       1.00      1.00      1.00         4
      Mango1       0.93      0.74      0.82        19
      Mango2       0.65      0.88      0.75        17
      Mango3       0.95      1.00      0.97        18
      Mango4       1.00      1.00      1.00        12
      Mango5       0.50      0.50      0.50         2
      Mango6       1.00      1.00      1.00         5
     Orange1       0.90      0.96      0.93        27
     Orange2       1.00      1.00      1.00        20
     Orange3       1.00      0.73      0.85        15
     Orange4       1.00      0.88      0.93         8
     Orange5       1.00      1.00      1.00         5

    accuracy                           0.91       176
   macro avg       0.93      0.91      0.91       176
weighte

In [ ]:
# SVN

rf_clf.fit(train_snv, train_set['Fruit Type'])
norm_pred = rf_clf.predict(val_snv)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.9488636363636364
              precision    recall  f1-score   support

      Apple1       1.00      1.00      1.00        19
      Apple2       1.00      1.00      1.00         3
      Apple3       1.00      1.00      1.00         2
      Apple4       1.00      1.00      1.00         4
      Mango1       0.88      0.74      0.80        19
      Mango2       0.76      0.94      0.84        17
      Mango3       1.00      1.00      1.00        18
      Mango4       1.00      0.92      0.96        12
      Mango5       0.67      1.00      0.80         2
      Mango6       1.00      1.00      1.00         5
     Orange1       0.96      1.00      0.98        27
     Orange2       1.00      1.00      1.00        20
     Orange3       1.00      0.87      0.93        15
     Orange4       1.00      1.00      1.00         8
     Orange5       1.00      1.00      1.00         5

    accuracy                           0.95       176
   macro avg       0.95      0.96      0.95       176
weighte

In [ ]:
# SAVGOL

rf_clf.fit(train_savgol, train_set['Fruit Type'])
norm_pred = rf_clf.predict(val_savgol)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.8863636363636364
              precision    recall  f1-score   support

      Apple1       1.00      1.00      1.00        19
      Apple2       1.00      1.00      1.00         3
      Apple3       1.00      1.00      1.00         2
      Apple4       0.80      1.00      0.89         4
      Mango1       0.93      0.74      0.82        19
      Mango2       0.64      0.82      0.72        17
      Mango3       0.81      0.94      0.87        18
      Mango4       1.00      0.83      0.91        12
      Mango5       0.33      0.50      0.40         2
      Mango6       1.00      1.00      1.00         5
     Orange1       0.96      0.96      0.96        27
     Orange2       0.90      0.95      0.93        20
     Orange3       0.91      0.67      0.77        15
     Orange4       1.00      0.88      0.93         8
     Orange5       1.00      1.00      1.00         5

    accuracy                           0.89       176
   macro avg       0.89      0.89      0.88       176
weighte

### Decision Tree classifier

In [ ]:
tree_clf = DecisionTreeClassifier(random_state=42)

# Normalizer
tree_clf.fit(train_norm, train_set['Fruit Type'])
norm_pred = tree_clf.predict(val_norm)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.8409090909090909
              precision    recall  f1-score   support

      Apple1       0.86      0.95      0.90        19
      Apple2       1.00      1.00      1.00         3
      Apple3       1.00      1.00      1.00         2
      Apple4       1.00      0.75      0.86         4
      Mango1       0.75      0.63      0.69        19
      Mango2       0.68      0.88      0.77        17
      Mango3       0.90      1.00      0.95        18
      Mango4       0.90      0.75      0.82        12
      Mango5       0.67      1.00      0.80         2
      Mango6       1.00      0.80      0.89         5
     Orange1       0.92      0.81      0.86        27
     Orange2       0.90      0.95      0.93        20
     Orange3       0.71      0.67      0.69        15
     Orange4       0.78      0.88      0.82         8
     Orange5       1.00      0.80      0.89         5

    accuracy                           0.84       176
   macro avg       0.87      0.86      0.86       176
weighte

In [ ]:
# Minmax Normalizer

tree_clf.fit(train_minmax, train_set['Fruit Type'])
norm_pred = tree_clf.predict(val_minmax)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.7329545454545454
              precision    recall  f1-score   support

      Apple1       0.76      0.68      0.72        19
      Apple2       1.00      1.00      1.00         3
      Apple3       1.00      1.00      1.00         2
      Apple4       0.60      0.75      0.67         4
      Mango1       0.68      0.68      0.68        19
      Mango2       0.60      0.71      0.65        17
      Mango3       0.76      0.89      0.82        18
      Mango4       0.73      0.67      0.70        12
      Mango5       0.17      0.50      0.25         2
      Mango6       0.57      0.80      0.67         5
     Orange1       0.83      0.70      0.76        27
     Orange2       0.81      0.85      0.83        20
     Orange3       1.00      0.53      0.70        15
     Orange4       0.86      0.75      0.80         8
     Orange5       0.67      0.80      0.73         5

    accuracy                           0.73       176
   macro avg       0.74      0.75      0.73       176
weighte

In [ ]:
# SVN

tree_clf.fit(train_snv, train_set['Fruit Type'])
norm_pred = tree_clf.predict(val_snv)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.8409090909090909
              precision    recall  f1-score   support

      Apple1       0.94      0.84      0.89        19
      Apple2       0.50      0.67      0.57         3
      Apple3       1.00      1.00      1.00         2
      Apple4       1.00      0.50      0.67         4
      Mango1       0.64      0.74      0.68        19
      Mango2       0.75      0.71      0.73        17
      Mango3       0.95      1.00      0.97        18
      Mango4       0.67      0.83      0.74        12
      Mango5       0.50      0.50      0.50         2
      Mango6       1.00      1.00      1.00         5
     Orange1       0.89      0.93      0.91        27
     Orange2       0.90      0.90      0.90        20
     Orange3       1.00      0.73      0.85        15
     Orange4       1.00      1.00      1.00         8
     Orange5       0.80      0.80      0.80         5

    accuracy                           0.84       176
   macro avg       0.84      0.81      0.81       176
weighte

In [ ]:
# SAVGOL

tree_clf.fit(train_savgol, train_set['Fruit Type'])
norm_pred = tree_clf.predict(val_savgol)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.7670454545454546
              precision    recall  f1-score   support

      Apple1       0.68      0.79      0.73        19
      Apple2       1.00      1.00      1.00         3
      Apple3       1.00      1.00      1.00         2
      Apple4       0.43      0.75      0.55         4
      Mango1       0.69      0.58      0.63        19
      Mango2       0.65      0.65      0.65        17
      Mango3       0.80      0.89      0.84        18
      Mango4       0.73      0.67      0.70        12
      Mango5       0.50      1.00      0.67         2
      Mango6       0.83      1.00      0.91         5
     Orange1       0.88      0.85      0.87        27
     Orange2       0.88      0.75      0.81        20
     Orange3       0.91      0.67      0.77        15
     Orange4       0.75      0.75      0.75         8
     Orange5       0.83      1.00      0.91         5

    accuracy                           0.77       176
   macro avg       0.77      0.82      0.78       176
weighte

### SVC

In [ ]:
svc_clf = SVC(random_state=42)

# Normalizer
svc_clf.fit(train_norm, train_set['Fruit Type'])
norm_pred = svc_clf.predict(val_norm)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.32386363636363635
              precision    recall  f1-score   support

      Apple1       0.18      0.63      0.29        19
      Apple2       0.00      0.00      0.00         3
      Apple3       0.00      0.00      0.00         2
      Apple4       0.00      0.00      0.00         4
      Mango1       0.50      0.26      0.34        19
      Mango2       0.38      0.65      0.48        17
      Mango3       0.46      0.61      0.52        18
      Mango4       0.00      0.00      0.00        12
      Mango5       0.00      0.00      0.00         2
      Mango6       0.00      0.00      0.00         5
     Orange1       0.38      0.48      0.43        27
     Orange2       0.36      0.25      0.29        20
     Orange3       0.00      0.00      0.00        15
     Orange4       0.00      0.00      0.00         8
     Orange5       0.00      0.00      0.00         5

    accuracy                           0.32       176
   macro avg       0.15      0.19      0.16       176
weight

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Minmax Normalizer

svc_clf.fit(train_minmax, train_set['Fruit Type'])
norm_pred = svc_clf.predict(val_minmax)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.48295454545454547
              precision    recall  f1-score   support

      Apple1       0.44      0.63      0.52        19
      Apple2       0.00      0.00      0.00         3
      Apple3       1.00      0.50      0.67         2
      Apple4       0.00      0.00      0.00         4
      Mango1       0.78      0.37      0.50        19
      Mango2       0.36      0.47      0.41        17
      Mango3       0.50      0.61      0.55        18
      Mango4       0.80      0.67      0.73        12
      Mango5       0.00      0.00      0.00         2
      Mango6       0.00      0.00      0.00         5
     Orange1       0.65      0.48      0.55        27
     Orange2       0.52      0.60      0.56        20
     Orange3       0.22      0.53      0.31        15
     Orange4       1.00      0.62      0.77         8
     Orange5       0.00      0.00      0.00         5

    accuracy                           0.48       176
   macro avg       0.42      0.37      0.37       176
weight

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# SVN

svc_clf.fit(train_snv, train_set['Fruit Type'])
norm_pred = svc_clf.predict(val_snv)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.2897727272727273
              precision    recall  f1-score   support

      Apple1       0.80      0.21      0.33        19
      Apple2       0.00      0.00      0.00         3
      Apple3       0.00      0.00      0.00         2
      Apple4       0.00      0.00      0.00         4
      Mango1       0.28      0.42      0.33        19
      Mango2       0.28      0.47      0.35        17
      Mango3       0.33      0.61      0.43        18
      Mango4       0.00      0.00      0.00        12
      Mango5       0.00      0.00      0.00         2
      Mango6       0.00      0.00      0.00         5
     Orange1       0.34      0.37      0.36        27
     Orange2       0.20      0.50      0.28        20
     Orange3       0.00      0.00      0.00        15
     Orange4       0.00      0.00      0.00         8
     Orange5       0.00      0.00      0.00         5

    accuracy                           0.29       176
   macro avg       0.15      0.17      0.14       176
weighte

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# SAVGOL

svc_clf.fit(train_savgol, train_set['Fruit Type'])
norm_pred = svc_clf.predict(val_savgol)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.25
              precision    recall  f1-score   support

      Apple1       0.50      0.32      0.39        19
      Apple2       0.00      0.00      0.00         3
      Apple3       0.00      0.00      0.00         2
      Apple4       0.00      0.00      0.00         4
      Mango1       0.50      0.05      0.10        19
      Mango2       0.11      0.24      0.15        17
      Mango3       0.18      0.67      0.28        18
      Mango4       0.50      0.42      0.45        12
      Mango5       0.00      0.00      0.00         2
      Mango6       0.00      0.00      0.00         5
     Orange1       0.33      0.41      0.37        27
     Orange2       0.00      0.00      0.00        20
     Orange3       0.00      0.00      0.00        15
     Orange4       0.62      0.62      0.62         8
     Orange5       0.00      0.00      0.00         5

    accuracy                           0.25       176
   macro avg       0.18      0.18      0.16       176
weighted avg       0.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### GaussianNB

In [ ]:
gb_clf = GaussianNB()

# Normalizer
gb_clf.fit(train_norm, train_set['Fruit Type'])
norm_pred = gb_clf.predict(val_norm)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.3125
              precision    recall  f1-score   support

      Apple1       0.33      0.05      0.09        19
      Apple2       0.21      1.00      0.35         3
      Apple3       1.00      0.50      0.67         2
      Apple4       0.44      1.00      0.62         4
      Mango1       1.00      0.21      0.35        19
      Mango2       0.17      0.06      0.09        17
      Mango3       0.45      0.50      0.47        18
      Mango4       0.27      0.50      0.35        12
      Mango5       0.00      0.00      0.00         2
      Mango6       0.25      0.40      0.31         5
     Orange1       0.82      0.33      0.47        27
     Orange2       0.56      0.25      0.34        20
     Orange3       0.11      0.20      0.14        15
     Orange4       0.20      0.62      0.30         8
     Orange5       0.22      0.40      0.29         5

    accuracy                           0.31       176
   macro avg       0.40      0.40      0.32       176
weighted avg       

In [ ]:
# Minmax Normalizer

gb_clf.fit(train_minmax, train_set['Fruit Type'])
norm_pred = gb_clf.predict(val_minmax)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.17045454545454544
              precision    recall  f1-score   support

      Apple1       0.00      0.00      0.00        19
      Apple2       0.00      0.00      0.00         3
      Apple3       0.25      1.00      0.40         2
      Apple4       0.33      1.00      0.50         4
      Mango1       0.17      0.05      0.08        19
      Mango2       0.00      0.00      0.00        17
      Mango3       0.38      0.33      0.35        18
      Mango4       0.18      0.50      0.27        12
      Mango5       0.00      0.00      0.00         2
      Mango6       0.10      0.40      0.16         5
     Orange1       0.00      0.00      0.00        27
     Orange2       0.00      0.00      0.00        20
     Orange3       0.16      0.33      0.21        15
     Orange4       0.22      0.25      0.24         8
     Orange5       0.11      0.40      0.17         5

    accuracy                           0.17       176
   macro avg       0.13      0.28      0.16       176
weight

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# SVN

gb_clf.fit(train_snv, train_set['Fruit Type'])
norm_pred = gb_clf.predict(val_snv)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.3806818181818182
              precision    recall  f1-score   support

      Apple1       0.57      0.21      0.31        19
      Apple2       1.00      1.00      1.00         3
      Apple3       0.67      1.00      0.80         2
      Apple4       1.00      0.75      0.86         4
      Mango1       0.75      0.32      0.44        19
      Mango2       0.27      0.18      0.21        17
      Mango3       0.20      0.22      0.21        18
      Mango4       0.31      0.33      0.32        12
      Mango5       0.00      0.00      0.00         2
      Mango6       1.00      1.00      1.00         5
     Orange1       0.44      0.44      0.44        27
     Orange2       0.06      0.05      0.05        20
     Orange3       0.22      0.47      0.30        15
     Orange4       0.40      1.00      0.57         8
     Orange5       0.71      1.00      0.83         5

    accuracy                           0.38       176
   macro avg       0.51      0.53      0.49       176
weighte

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# SAVGOL

gb_clf.fit(train_savgol, train_set['Fruit Type'])
norm_pred = gb_clf.predict(val_savgol)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.18181818181818182
              precision    recall  f1-score   support

      Apple1       0.00      0.00      0.00        19
      Apple2       0.00      0.00      0.00         3
      Apple3       0.40      1.00      0.57         2
      Apple4       0.36      1.00      0.53         4
      Mango1       0.17      0.05      0.08        19
      Mango2       0.00      0.00      0.00        17
      Mango3       0.24      0.28      0.26        18
      Mango4       0.27      0.50      0.35        12
      Mango5       0.00      0.00      0.00         2
      Mango6       0.08      0.40      0.14         5
     Orange1       0.00      0.00      0.00        27
     Orange2       0.50      0.10      0.17        20
     Orange3       0.14      0.20      0.16        15
     Orange4       0.38      0.62      0.48         8
     Orange5       0.11      0.40      0.17         5

    accuracy                           0.18       176
   macro avg       0.18      0.30      0.19       176
weight

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Linear Discriminant Analysis

In [ ]:
ld_clf = LinearDiscriminantAnalysis()

# Normalizer
ld_clf.fit(train_norm, train_set['Fruit Type'])
norm_pred = ld_clf.predict(val_norm)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.9488636363636364
              precision    recall  f1-score   support

      Apple1       1.00      0.95      0.97        19
      Apple2       1.00      1.00      1.00         3
      Apple3       1.00      1.00      1.00         2
      Apple4       1.00      1.00      1.00         4
      Mango1       0.82      0.74      0.78        19
      Mango2       0.75      0.88      0.81        17
      Mango3       1.00      0.94      0.97        18
      Mango4       0.92      1.00      0.96        12
      Mango5       1.00      1.00      1.00         2
      Mango6       1.00      1.00      1.00         5
     Orange1       1.00      1.00      1.00        27
     Orange2       1.00      1.00      1.00        20
     Orange3       1.00      1.00      1.00        15
     Orange4       1.00      1.00      1.00         8
     Orange5       1.00      1.00      1.00         5

    accuracy                           0.95       176
   macro avg       0.97      0.97      0.97       176
weighte

In [ ]:
# Minmax Normalizer

ld_clf.fit(train_minmax, train_set['Fruit Type'])
norm_pred = ld_clf.predict(val_minmax)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.9545454545454546
              precision    recall  f1-score   support

      Apple1       1.00      0.95      0.97        19
      Apple2       1.00      1.00      1.00         3
      Apple3       1.00      1.00      1.00         2
      Apple4       1.00      1.00      1.00         4
      Mango1       0.93      0.68      0.79        19
      Mango2       0.73      0.94      0.82        17
      Mango3       1.00      1.00      1.00        18
      Mango4       1.00      1.00      1.00        12
      Mango5       0.67      1.00      0.80         2
      Mango6       1.00      1.00      1.00         5
     Orange1       1.00      1.00      1.00        27
     Orange2       1.00      1.00      1.00        20
     Orange3       1.00      1.00      1.00        15
     Orange4       1.00      1.00      1.00         8
     Orange5       1.00      1.00      1.00         5

    accuracy                           0.95       176
   macro avg       0.95      0.97      0.96       176
weighte

In [ ]:
# SVN

ld_clf.fit(train_snv, train_set['Fruit Type'])
norm_pred = ld_clf.predict(val_snv)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.9375
              precision    recall  f1-score   support

      Apple1       1.00      0.84      0.91        19
      Apple2       1.00      1.00      1.00         3
      Apple3       1.00      1.00      1.00         2
      Apple4       1.00      1.00      1.00         4
      Mango1       0.88      0.74      0.80        19
      Mango2       0.73      0.94      0.82        17
      Mango3       1.00      0.89      0.94        18
      Mango4       1.00      1.00      1.00        12
      Mango5       1.00      1.00      1.00         2
      Mango6       1.00      1.00      1.00         5
     Orange1       0.96      1.00      0.98        27
     Orange2       0.95      1.00      0.98        20
     Orange3       1.00      1.00      1.00        15
     Orange4       0.89      1.00      0.94         8
     Orange5       1.00      1.00      1.00         5

    accuracy                           0.94       176
   macro avg       0.96      0.96      0.96       176
weighted avg       

In [ ]:
# SAVGOL

ld_clf.fit(train_savgol, train_set['Fruit Type'])
norm_pred = ld_clf.predict(val_savgol)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.9659090909090909
              precision    recall  f1-score   support

      Apple1       1.00      1.00      1.00        19
      Apple2       1.00      1.00      1.00         3
      Apple3       1.00      1.00      1.00         2
      Apple4       1.00      1.00      1.00         4
      Mango1       1.00      0.68      0.81        19
      Mango2       0.74      1.00      0.85        17
      Mango3       1.00      1.00      1.00        18
      Mango4       1.00      1.00      1.00        12
      Mango5       1.00      1.00      1.00         2
      Mango6       1.00      1.00      1.00         5
     Orange1       1.00      1.00      1.00        27
     Orange2       1.00      1.00      1.00        20
     Orange3       1.00      1.00      1.00        15
     Orange4       1.00      1.00      1.00         8
     Orange5       1.00      1.00      1.00         5

    accuracy                           0.97       176
   macro avg       0.98      0.98      0.98       176
weighte

### LogisticRegression

In [ ]:
lr_clf = LogisticRegression(max_iter=1000, random_state=42, solver='newton-cg')

# Normalizer
lr_clf.fit(train_norm, train_set['Fruit Type'])
norm_pred = lr_clf.predict(val_norm)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.2556818181818182
              precision    recall  f1-score   support

      Apple1       0.00      0.00      0.00        19
      Apple2       0.00      0.00      0.00         3
      Apple3       0.00      0.00      0.00         2
      Apple4       0.00      0.00      0.00         4
      Mango1       0.15      0.21      0.17        19
      Mango2       0.26      0.47      0.33        17
      Mango3       0.28      0.83      0.42        18
      Mango4       0.00      0.00      0.00        12
      Mango5       0.00      0.00      0.00         2
      Mango6       0.00      0.00      0.00         5
     Orange1       0.33      0.48      0.39        27
     Orange2       0.24      0.25      0.24        20
     Orange3       0.00      0.00      0.00        15
     Orange4       0.00      0.00      0.00         8
     Orange5       0.00      0.00      0.00         5

    accuracy                           0.26       176
   macro avg       0.08      0.15      0.10       176
weighte

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Minmax Normalizer

lr_clf.fit(train_minmax, train_set['Fruit Type'])
norm_pred = lr_clf.predict(val_minmax)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.42045454545454547
              precision    recall  f1-score   support

      Apple1       0.67      0.42      0.52        19
      Apple2       0.00      0.00      0.00         3
      Apple3       1.00      1.00      1.00         2
      Apple4       1.00      0.50      0.67         4
      Mango1       0.20      0.21      0.21        19
      Mango2       0.31      0.65      0.42        17
      Mango3       0.38      0.67      0.48        18
      Mango4       1.00      0.42      0.59        12
      Mango5       0.00      0.00      0.00         2
      Mango6       1.00      0.20      0.33         5
     Orange1       0.43      0.37      0.40        27
     Orange2       0.31      0.50      0.38        20
     Orange3       1.00      0.07      0.12        15
     Orange4       0.73      1.00      0.84         8
     Orange5       0.00      0.00      0.00         5

    accuracy                           0.42       176
   macro avg       0.54      0.40      0.40       176
weight

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# SVN

lr_clf.fit(train_snv, train_set['Fruit Type'])
norm_pred = lr_clf.predict(val_snv)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.6647727272727273
              precision    recall  f1-score   support

      Apple1       0.57      0.42      0.48        19
      Apple2       1.00      0.33      0.50         3
      Apple3       1.00      1.00      1.00         2
      Apple4       1.00      0.75      0.86         4
      Mango1       0.53      0.53      0.53        19
      Mango2       0.52      0.76      0.62        17
      Mango3       0.64      1.00      0.78        18
      Mango4       0.89      0.67      0.76        12
      Mango5       1.00      0.50      0.67         2
      Mango6       1.00      1.00      1.00         5
     Orange1       0.69      0.41      0.51        27
     Orange2       0.62      0.75      0.68        20
     Orange3       0.67      0.80      0.73        15
     Orange4       0.89      1.00      0.94         8
     Orange5       1.00      0.40      0.57         5

    accuracy                           0.66       176
   macro avg       0.80      0.69      0.71       176
weighte

In [ ]:
# SAVGOL

lr_clf.fit(train_savgol, train_set['Fruit Type'])
norm_pred = lr_clf.predict(val_savgol)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.29545454545454547
              precision    recall  f1-score   support

      Apple1       0.00      0.00      0.00        19
      Apple2       0.00      0.00      0.00         3
      Apple3       1.00      1.00      1.00         2
      Apple4       0.00      0.00      0.00         4
      Mango1       0.14      0.16      0.15        19
      Mango2       0.22      0.53      0.31        17
      Mango3       0.38      0.83      0.53        18
      Mango4       0.25      0.08      0.12        12
      Mango5       0.00      0.00      0.00         2
      Mango6       0.00      0.00      0.00         5
     Orange1       0.36      0.37      0.36        27
     Orange2       0.37      0.35      0.36        20
     Orange3       0.00      0.00      0.00        15
     Orange4       0.56      0.62      0.59         8
     Orange5       0.00      0.00      0.00         5

    accuracy                           0.30       176
   macro avg       0.22      0.26      0.23       176
weight

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### LinearSVC

In [ ]:
lsvc = LinearSVC(max_iter=2000)

# Normalizer
lsvc.fit(train_norm, train_set['Fruit Type'])
norm_pred = lsvc.predict(val_norm)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.2897727272727273
              precision    recall  f1-score   support

      Apple1       0.33      0.05      0.09        19
      Apple2       0.00      0.00      0.00         3
      Apple3       0.00      0.00      0.00         2
      Apple4       0.00      0.00      0.00         4
      Mango1       0.26      0.26      0.26        19
      Mango2       0.28      0.65      0.39        17
      Mango3       0.30      0.83      0.44        18
      Mango4       0.00      0.00      0.00        12
      Mango5       0.00      0.00      0.00         2
      Mango6       0.00      0.00      0.00         5
     Orange1       0.37      0.48      0.42        27
     Orange2       0.21      0.30      0.25        20
     Orange3       0.00      0.00      0.00        15
     Orange4       0.00      0.00      0.00         8
     Orange5       0.00      0.00      0.00         5

    accuracy                           0.29       176
   macro avg       0.12      0.17      0.12       176
weighte

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Minmax Normalizer

lsvc.fit(train_minmax, train_set['Fruit Type'])
norm_pred = lsvc.predict(val_minmax)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.7556818181818182
              precision    recall  f1-score   support

      Apple1       0.91      0.53      0.67        19
      Apple2       1.00      0.33      0.50         3
      Apple3       1.00      1.00      1.00         2
      Apple4       1.00      1.00      1.00         4
      Mango1       0.70      0.37      0.48        19
      Mango2       0.53      1.00      0.69        17
      Mango3       0.62      1.00      0.77        18
      Mango4       1.00      1.00      1.00        12
      Mango5       1.00      0.50      0.67         2
      Mango6       1.00      1.00      1.00         5
     Orange1       0.84      0.59      0.70        27
     Orange2       0.69      0.90      0.78        20
     Orange3       0.83      0.67      0.74        15
     Orange4       1.00      1.00      1.00         8
     Orange5       1.00      0.80      0.89         5

    accuracy                           0.76       176
   macro avg       0.88      0.78      0.79       176
weighte

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [ ]:
# SVN

lsvc.fit(train_snv, train_set['Fruit Type'])
norm_pred = lsvc.predict(val_snv)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.9204545454545454
              precision    recall  f1-score   support

      Apple1       0.94      0.84      0.89        19
      Apple2       1.00      1.00      1.00         3
      Apple3       1.00      1.00      1.00         2
      Apple4       1.00      1.00      1.00         4
      Mango1       0.93      0.68      0.79        19
      Mango2       0.71      1.00      0.83        17
      Mango3       0.89      0.94      0.92        18
      Mango4       1.00      1.00      1.00        12
      Mango5       1.00      1.00      1.00         2
      Mango6       1.00      1.00      1.00         5
     Orange1       1.00      0.85      0.92        27
     Orange2       0.87      1.00      0.93        20
     Orange3       1.00      1.00      1.00        15
     Orange4       1.00      1.00      1.00         8
     Orange5       1.00      1.00      1.00         5

    accuracy                           0.92       176
   macro avg       0.96      0.95      0.95       176
weighte

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [ ]:
# SAVGOL

lsvc.fit(train_savgol, train_set['Fruit Type'])
norm_pred = lsvc.predict(val_savgol)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

0.3693181818181818
              precision    recall  f1-score   support

      Apple1       0.36      0.21      0.27        19
      Apple2       0.00      0.00      0.00         3
      Apple3       0.67      1.00      0.80         2
      Apple4       1.00      0.50      0.67         4
      Mango1       0.18      0.21      0.20        19
      Mango2       0.23      0.59      0.33        17
      Mango3       0.38      0.89      0.53        18
      Mango4       0.50      0.33      0.40        12
      Mango5       0.00      0.00      0.00         2
      Mango6       1.00      0.60      0.75         5
     Orange1       0.65      0.41      0.50        27
     Orange2       0.27      0.20      0.23        20
     Orange3       0.00      0.00      0.00        15
     Orange4       0.56      0.62      0.59         8
     Orange5       0.00      0.00      0.00         5

    accuracy                           0.37       176
   macro avg       0.39      0.37      0.35       176
weighte

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `

### Deep Learning with Dense Layers

In [18]:

model = tf.keras.Sequential([
tf.keras.layers.Dense(500, input_dim=train_wave.shape[1] ,activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)),
  tf.keras.layers.Dense(800, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)),
  tf.keras.layers.Dense(400, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)),
  tf.keras.layers.Dense(250, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)),
  tf.keras.layers.Dense(150, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)),
  tf.keras.layers.Dense(50, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.02)),
  tf.keras.layers.Dense(15, activation='softmax', kernel_regularizer=tf.keras.regularizers.L2(0.01))
])
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor="loss",
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=1e-2,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=500,
        verbose=1,
    )
]
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 500)               201000    
                                                                 
 dense_1 (Dense)             (None, 800)               400800    
                                                                 
 dense_2 (Dense)             (None, 400)               320400    
                                                                 
 dense_3 (Dense)             (None, 250)               100250    
                                                                 
 dense_4 (Dense)             (None, 150)               37650     
                                                                 
 dense_5 (Dense)             (None, 50)                7550      
                                                                 
 dense_6 (Dense)             (None, 15)                7

In [ ]:
# Normalized

num_epochs = 1000
#  Train the model
model.fit(train_norm, dummy_y, epochs=num_epochs, batch_size=128, callbacks=callbacks, validation_data=(val_norm, val_encoder_y))

Epoch 1/1000
6/6 [==============================] - 1s 41ms/step - loss: 2.6741 - accuracy: 0.1181 - val_loss: 2.5904 - val_accuracy: 0.1705
Epoch 2/1000
6/6 [==============================] - 0s 12ms/step - loss: 2.5410 - accuracy: 0.1892 - val_loss: 2.4653 - val_accuracy: 0.1818
Epoch 3/1000
6/6 [==============================] - 0s 7ms/step - loss: 2.4711 - accuracy: 0.1693 - val_loss: 2.3940 - val_accuracy: 0.1932
Epoch 4/1000
6/6 [==============================] - 0s 8ms/step - loss: 2.4233 - accuracy: 0.2077 - val_loss: 2.3313 - val_accuracy: 0.1875
Epoch 5/1000
6/6 [==============================] - 0s 8ms/step - loss: 2.3930 - accuracy: 0.1693 - val_loss: 2.3086 - val_accuracy: 0.1989
Epoch 6/1000
6/6 [==============================] - 0s 10ms/step - loss: 2.3660 - accuracy: 0.2191 - val_loss: 2.2878 - val_accuracy: 0.2102
Epoch 7/1000
6/6 [==============================] - 0s 8ms/step - loss: 2.3386 - accuracy: 0.2148 - val_loss: 2.2617 - val_accuracy: 0.2045
Epoch 8/1000
6/6 

In [ ]:
predict = model.evaluate(val_norm, val_encoder_y)

6/6 [==============================] - 0s 3ms/step - loss: 0.7169 - accuracy: 0.8295


In [ ]:
predict

[0.7168596386909485, 0.8295454382896423]

In [ ]:
# Normalized minmax

#  Train the model
model.fit(train_minmax, dummy_y, epochs=num_epochs, callbacks=callbacks, batch_size=128,validation_data=(val_minmax, val_encoder_y))

Epoch 1/1000
6/6 [==============================] - 0s 14ms/step - loss: 196.8771 - accuracy: 0.1707 - val_loss: 46.3141 - val_accuracy: 0.1477
Epoch 2/1000
6/6 [==============================] - 0s 8ms/step - loss: 24.2484 - accuracy: 0.2048 - val_loss: 8.2531 - val_accuracy: 0.1477
Epoch 3/1000
6/6 [==============================] - 0s 8ms/step - loss: 5.1476 - accuracy: 0.1394 - val_loss: 2.7405 - val_accuracy: 0.1477
Epoch 4/1000
6/6 [==============================] - 0s 9ms/step - loss: 2.6624 - accuracy: 0.1550 - val_loss: 2.6937 - val_accuracy: 0.1648
Epoch 5/1000
6/6 [==============================] - 0s 8ms/step - loss: 2.4632 - accuracy: 0.1991 - val_loss: 2.2812 - val_accuracy: 0.2273
Epoch 6/1000
6/6 [==============================] - 0s 8ms/step - loss: 2.2545 - accuracy: 0.2688 - val_loss: 2.1951 - val_accuracy: 0.2614
Epoch 7/1000
6/6 [==============================] - 0s 7ms/step - loss: 2.1573 - accuracy: 0.2575 - val_loss: 2.0451 - val_accuracy: 0.3011
Epoch 8/1000
6/

In [ ]:
# SNV

num_epochs = 2000
#  Train the model
model.fit(train_snv, dummy_y, epochs=num_epochs,callbacks=callbacks, batch_size=128,validation_data=(val_snv, val_encoder_y))

Epoch 1/2000
6/6 [==============================] - 0s 33ms/step - loss: 336.1497 - accuracy: 0.1038 - val_loss: 84.4080 - val_accuracy: 0.0909
Epoch 2/2000
6/6 [==============================] - 0s 29ms/step - loss: 50.2949 - accuracy: 0.1166 - val_loss: 23.2060 - val_accuracy: 0.1193
Epoch 3/2000
6/6 [==============================] - 0s 27ms/step - loss: 12.3921 - accuracy: 0.1067 - val_loss: 3.1317 - val_accuracy: 0.0682
Epoch 4/2000
6/6 [==============================] - 0s 29ms/step - loss: 2.8607 - accuracy: 0.0996 - val_loss: 2.6943 - val_accuracy: 0.1080
Epoch 5/2000
6/6 [==============================] - 0s 26ms/step - loss: 2.7304 - accuracy: 0.1124 - val_loss: 2.6897 - val_accuracy: 0.1080
Epoch 6/2000
6/6 [==============================] - 0s 27ms/step - loss: 2.7072 - accuracy: 0.1152 - val_loss: 2.6844 - val_accuracy: 0.1080
Epoch 7/2000
6/6 [==============================] - 0s 25ms/step - loss: 2.6953 - accuracy: 0.1152 - val_loss: 2.6723 - val_accuracy: 0.1080
Epoch 8

In [ ]:
# SAVGOL

#  Train the model
model.fit(train_savgol, dummy_y, epochs=num_epochs, callbacks=callbacks, batch_size=128,validation_data=(val_savgol, val_encoder_y))

Epoch 1/2000
6/6 [==============================] - 0s 20ms/step - loss: 9.7451 - accuracy: 0.0953 - val_loss: 5.5404 - val_accuracy: 0.0966
Epoch 2/2000
6/6 [==============================] - 0s 10ms/step - loss: 4.9209 - accuracy: 0.1195 - val_loss: 3.6354 - val_accuracy: 0.1648
Epoch 3/2000
6/6 [==============================] - 0s 9ms/step - loss: 3.0026 - accuracy: 0.1380 - val_loss: 2.5257 - val_accuracy: 0.2216
Epoch 4/2000
6/6 [==============================] - 0s 9ms/step - loss: 2.4186 - accuracy: 0.1991 - val_loss: 2.4047 - val_accuracy: 0.1989
Epoch 5/2000
6/6 [==============================] - 0s 9ms/step - loss: 2.3177 - accuracy: 0.2006 - val_loss: 2.3863 - val_accuracy: 0.2330
Epoch 6/2000
6/6 [==============================] - 0s 10ms/step - loss: 2.2067 - accuracy: 0.2532 - val_loss: 2.2009 - val_accuracy: 0.2898
Epoch 7/2000
6/6 [==============================] - 0s 11ms/step - loss: 2.1115 - accuracy: 0.2575 - val_loss: 2.1280 - val_accuracy: 0.2557
Epoch 8/2000
6/6

### Deep Learning with Sparse Categorical Cross Entropy

In [ ]:
# Changing the outputs 


from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()
train_label = label.fit_transform(train_set['Fruit Type'])
val_label = label.transform(val_set['Fruit Type'])
val_label

array([13, 11,  6, 12, 11,  8,  5,  0,  6,  0,  0,  1, 14, 13,  5,  0, 10,
        5,  4, 12, 11,  3,  6, 10,  0,  8,  6, 10, 11, 10,  0,  6, 10,  0,
        5,  9, 12, 10,  6, 10,  4,  4, 12,  5,  4,  4, 10,  4, 12, 10,  4,
       10,  7,  5, 14, 10,  0,  5, 11, 13,  6, 11, 12,  7, 11, 10,  4,  0,
        3, 13,  6,  5, 11, 10,  4,  3,  0, 11,  2,  9,  4,  6,  0,  5, 11,
        5,  4,  7, 14,  3, 14, 13,  7, 12,  5, 13, 13,  7, 11, 11, 10,  4,
        4,  6, 10, 12, 10,  9, 10,  6,  9,  5,  4, 12,  4,  7,  6, 10,  6,
        5,  7,  0,  6,  7, 11, 12,  4,  1,  5, 14,  5,  6,  0,  0, 12,  9,
       10, 11,  4, 10,  1, 10, 12,  7,  5, 11, 12,  6,  0, 10,  6, 11,  0,
       13, 10, 11,  7,  7, 10,  0, 10,  0,  4, 12, 11, 11,  4,  2, 12,  0,
        7,  5, 11, 10,  6, 10])

In [19]:

model = tf.keras.Sequential([
tf.keras.layers.Dense(500, input_dim=train_wave.shape[1] ,activation='relu',kernel_regularizer=tf.keras.regularizers.L2(0.01)),
  tf.keras.layers.Dense(850, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)),
  tf.keras.layers.Dense(400, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)),
  tf.keras.layers.Dense(250, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)),
  tf.keras.layers.Dense(150, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)),
  tf.keras.layers.Dense(50, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)),
  tf.keras.layers.Dense(15, activation='linear', kernel_regularizer=tf.keras.regularizers.L2(0.01)),
])
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
)
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor="val_loss",
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=1e-2,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=30,
        verbose=1,
    )
]
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 500)               201000    
                                                                 
 dense_8 (Dense)             (None, 850)               425850    
                                                                 
 dense_9 (Dense)             (None, 400)               340400    
                                                                 
 dense_10 (Dense)            (None, 250)               100250    
                                                                 
 dense_11 (Dense)            (None, 150)               37650     
                                                                 
 dense_12 (Dense)            (None, 50)                7550      
                                                                 
 dense_13 (Dense)            (None, 15)               

In [ ]:
# Normalized

num_epochs = 2000
#  Train the model
model.fit(train_norm,train_label, epochs=num_epochs,callbacks=callbacks, validation_data=(val_norm, val_label))

In [ ]:
new_norm = tf.convert_to_tensor(train_norm)
pred = []
train_pred = model.predict(new_norm)
for i in range(new_norm.shape[0]):
  prediction_p = tf.nn.softmax(train_pred[i])
  yhat = np.argmax(prediction_p)
  pred.append(yhat)
new_pred = np.array(pred)
print(accuracy_score(train_label, new_pred))
# train_pred = model.predict(new_norm)
# prediction_p = tf.nn.softmax(train_pred)
# yhat = np.argmax(prediction_p)

In [ ]:
new_norm = tf.convert_to_tensor(val_norm)
pred = []
train_pred = model.predict(new_norm)
for i in range(new_norm.shape[0]):
  prediction_p = tf.nn.softmax(train_pred[i])
  yhat = np.argmax(prediction_p)
  pred.append(yhat)
new_pred = np.array(pred)
print(accuracy_score(val_label, new_pred))
# train_pred = model.predict(new_norm)
# prediction_p = tf.nn.softmax(train_pred)
# yhat = np.argmax(prediction_p)

In [ ]:
# Normalized Minmax

num_epochs = 2000
#  Train the model
model.fit(train_minmax,train_label, epochs=num_epochs, callbacks=callbacks, validation_data=(val_minmax, val_label))

In [ ]:
new_norm = tf.convert_to_tensor(train_minmax)
pred = []
train_pred = model.predict(new_norm)
for i in range(new_norm.shape[0]):
  prediction_p = tf.nn.softmax(train_pred[i])
  yhat = np.argmax(prediction_p)
  pred.append(yhat)
new_pred = np.array(pred)
print(accuracy_score(train_label, new_pred))
# train_pred = model.predict(new_norm)
# prediction_p = tf.nn.softmax(train_pred)
# yhat = np.argmax(prediction_p)

In [ ]:
new_norm = tf.convert_to_tensor(val_minmax)
pred = []
train_pred = model.predict(new_norm)
for i in range(new_norm.shape[0]):
  prediction_p = tf.nn.softmax(train_pred[i])
  yhat = np.argmax(prediction_p)
  pred.append(yhat)
new_pred = np.array(pred)
print(accuracy_score(val_label, new_pred))
# train_pred = model.predict(new_norm)
# prediction_p = tf.nn.softmax(train_pred)
# yhat = np.argmax(prediction_p)

In [ ]:
# SNV

num_epochs = 2500
#  Train the model
model.fit(train_snv,train_label, epochs=num_epochs, callbacks=callbacks, validation_data=(val_snv, val_label))

In [ ]:
new_norm = tf.convert_to_tensor(train_snv)
pred = []
train_pred = model.predict(new_norm)
for i in range(new_norm.shape[0]):
  prediction_p = tf.nn.softmax(train_pred[i])
  yhat = np.argmax(prediction_p)
  pred.append(yhat)
new_pred = np.array(pred)
print(accuracy_score(train_label, new_pred))
# train_pred = model.predict(new_norm)
# prediction_p = tf.nn.softmax(train_pred)
# yhat = np.argmax(prediction_p)

In [ ]:
new_norm = tf.convert_to_tensor(val_snv)
pred = []
train_pred = model.predict(new_norm)
for i in range(new_norm.shape[0]):
  prediction_p = tf.nn.softmax(train_pred[i])
  yhat = np.argmax(prediction_p)
  pred.append(yhat)
new_pred = np.array(pred)
print(accuracy_score(val_label, new_pred))
# train_pred = model.predict(new_norm)
# prediction_p = tf.nn.softmax(train_pred)
# yhat = np.argmax(prediction_p)

In [ ]:
# SAVGOL

num_epochs = 2000
#  Train the model
model.fit(train_savgol,train_label, epochs=num_epochs,callbacks=callbacks, validation_data=(val_savgol, val_label))

In [ ]:
new_norm = tf.convert_to_tensor(train_savgol)
pred = []
train_pred = model.predict(new_norm)
for i in range(new_norm.shape[0]):
  prediction_p = tf.nn.softmax(train_pred[i])
  yhat = np.argmax(prediction_p)
  pred.append(yhat)
new_pred = np.array(pred)
print(accuracy_score(train_label, new_pred))
# train_pred = model.predict(new_norm)
# prediction_p = tf.nn.softmax(train_pred)
# yhat = np.argmax(prediction_p)

In [ ]:
new_norm = tf.convert_to_tensor(val_savgol)
pred = []
train_pred = model.predict(new_norm)
for i in range(new_norm.shape[0]):
  prediction_p = tf.nn.softmax(train_pred[i])
  yhat = np.argmax(prediction_p)
  pred.append(yhat)
new_pred = np.array(pred)
print(accuracy_score(val_label, new_pred))
# train_pred = model.predict(new_norm)
# prediction_p = tf.nn.softmax(train_pred)
# yhat = np.argmax(prediction_p)

### XGBoostClassifier

In [ ]:
import xgboost
from xgboost import XGBClassifier

xgb = XGBClassifier(random_state=42)

# Normalizer
xgb.fit(train_norm, train_set['Fruit Type'])
norm_pred = xgb.predict(val_norm)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

In [ ]:
# Normalizer minmax

xgb.fit(train_minmax, train_set['Fruit Type'])
norm_pred = xgb.predict(val_minmax)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

In [ ]:
# SNV

xgb.fit(train_snv, train_set['Fruit Type'])
norm_pred = xgb.predict(val_snv)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

In [ ]:
# SAVGOL

xgb.fit(train_savgol, train_set['Fruit Type'])
norm_pred = xgb.predict(val_savgol)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

In [ ]:
import lightgbm

from lightgbm import LGBMClassifier

In [ ]:
lgbm = LGBMClassifier()

# Normalizer
lgbm.fit(train_norm, train_set['Fruit Type'])
norm_pred = lgbm.predict(val_norm)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

In [ ]:
# Normalizer minmax

lgbm.fit(train_minmax, train_set['Fruit Type'])
norm_pred = lgbm.predict(val_minmax)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

In [ ]:
# SNV

lgbm.fit(train_snv, train_set['Fruit Type'])
norm_pred = lgbm.predict(val_snv)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

In [ ]:
# SAVGOL

lgbm.fit(train_savgol, train_set['Fruit Type'])
norm_pred = lgbm.predict(val_savgol)
print(accuracy_score(val_set['Fruit Type'], norm_pred))
print(classification_report(val_set['Fruit Type'], norm_pred))

### Visualization

In [ ]:
plt.figure(figsize=(10.0, 10.0))
sns.lineplot(x="Daypoint", y="Apple1", data=df)

In [ ]:
df